# HV19.08 SmileNcryptor 4.0

We get an SQL dump - here are the relevant parts:
    
```sql
INSERT INTO `creditcards` VALUES 
(1,'Sirius Black',':)QVXSZUVY\ZYYZ[a','12/2020'),
(2,'Hermione Granger',':)QOUW[VT^VY]bZ_','04/2021'),
(3,'Draco Malfoy',':)SPPVSSYVV\YY_\\]','05/2020'),
(4,'Severus Snape',':)RPQRSTUVWXYZ[\]^','10/2020'),
(5,'Ron Weasley',':)QTVWRSVUXW[_Z`\b','11/2020');
```

```sql
INSERT INTO `flags` VALUES (1,'HV19{',':)SlQRUPXWVo\Vuv_n_\ajjce','}');
```

One weird thing here are the backslashes. When importing this data into MySQL, they get interpreted, so we end up with invalid data. However, that's a bug in the challenge. They are meant to be read literally. First thing which caused some confusion and ended up in me spending way too much time on this challenge...

Either way - let's get the data into Python so we can play with it. For starters, let's look at the length and character sets. I (correctly) assumed that `:)` wasn't part of the actual data, just a marker for the Smile "encryption".

In [2]:
ccs = {
    'sb': r'QVXSZUVY\ZYYZ[a',
    'hg': r'QOUW[VT^VY]bZ_',
    'dm': r'SPPVSSYVV\YY_\\]',
    'ss': r'RPQRSTUVWXYZ[\]^',
    'rw': r'QTVWRSVUXW[_Z`\b',
}
flag = r'SlQRUPXWVo\Vuv_n_\ajjce'

In [11]:
ccs_chars = set().union(*ccs.values())
print(f"credit cards: {len(ccs_chars)} chars: {''.join(ccs_chars)}")

chars = set(flag).union(*ccs.values())
print(f"total: {len(chars)} chars: {''.join(chars)}")

credit cards: 20 chars: PR`S_XVQ[a]T^OZ\UYbW
total: 28 chars: [PeYR`Sav]T^_OXjo\ZulVnQUcbW


In [13]:
print({k: len(v) for k, v in ccs.items()})

{'sb': 15, 'hg': 14, 'dm': 16, 'ss': 16, 'rw': 16}


Hmm. The length looks like a 1:1 mapping from characters to numbers, but we have 20 characters for 10 possible digits. Let's see if we can find out more about how credit card numbers are structured.

From [a website](https://www.creditcardinsider.com/learn/anatomy-of-a-credit-card/), we learn:

> The first digit is different for each card network:
> - Visa cards – Begin with a 4 and have 13 or 16 digits
> - Mastercard cards – Begin with a 5 and has 16 digits
> - American Express cards – Begin with a 3, followed by a 4 or a 7  has 15 digits
> - Discover cards – Begin with a 6 and have 16 digits
> - Diners Club and Carte Blanche cards – Begin with a 3, followed by a 0, 6, or 8 and have 14 digits

*Note that Wikipedia has a [more complete list](https://en.wikipedia.org/wiki/Payment_card_number#Issuer_identification_number_(IIN)) which I initially used. I later found out that the data in the challenge didn't match that list (or [Discover's official compliance document](https://www.discovernetwork.com/downloads/IPP_VAR_Compliance.pdf) for Diner's Club) because the 14-digit number starts with `30`, but only the `36`-range is allowed to have 14 characters. However, the numbers used are [official test numbers](https://www.paypalobjects.com/en_AU/vhelp/paypalmanager_help/credit_card_numbers.htm), so while this confused me and cost me a lot of time, I can't really blame the authors for that one. Anyways - for the sake of simplicity I'll use the simplified list.*

Also, there's the [Luhn Algorithm](https://en.wikipedia.org/wiki/Luhn_algorithm) which allows validating a credit card number. Let's ~~implement~~ steal it [from stackoverflow](https://stackoverflow.com/a/21079551):

In [14]:
def luhn_checksum(card_number):
    def digits_of(n):
        return [int(d) for d in str(n)]
    digits = digits_of(card_number)
    odd_digits = digits[-1::-2]
    even_digits = digits[-2::-2]
    checksum = 0
    checksum += sum(odd_digits)
    for d in even_digits:
        checksum += sum(digits_of(d*2))
    return checksum % 10

def is_luhn_valid(card_number):
    return luhn_checksum(card_number) == 0

What information can we infer about the credit card numbers? Based on the above list, we can infer that:

- Hermione's card is 14 digits long, so her card number starts with `3[068]`.
- Ron's card is 15 digits long, so it begins with `3[47]`.

Thus, we can assume that `Q = 3` holds. The ascii characters in the data all are relatively close to each other, so maybe it's just a simple offset? Let's try!

In [17]:
def decrypt(data):
    diff = ord('Q') - ord('3')  # 30
    decrypted = [chr(ord(c) - diff) for c in data]
    return ''.join(decrypted)

ccs_plain = {k: decrypt(v) for k, v in ccs.items()}
print(ccs_plain)

{'sb': '38:5<78;><;;<=C', 'hg': '3179=86@8;?D<A', 'dm': '522855;88>;;A>>?', 'ss': '423456789:;<=>?@', 'rw': '36894587:9=A<B>D'}


Nope. After some thinking (and talking to people who solved it) I had an idea: Maybe it's a [polyalphabetic cipher](https://en.wikipedia.org/wiki/Polyalphabetic_cipher) (e.g. [Vigenère](https://en.wikipedia.org/wiki/Vigen%C3%A8re_cipher)) where the difference shifts with every character? That'd also explain why the character set of the ciphertext is bigger than the one of the plaintext.

In [19]:
def decrypt(data):
    diff = ord('Q') - ord('3')  # 30
    decrypted = [chr(ord(c) - diff - i) for i, c in enumerate(data)]
    return ''.join(decrypted)

ccs_plain = {k: decrypt(v) for k, v in ccs.items()}
print(ccs_plain)

{'sb': '378282246310005', 'hg': '30569309025904', 'dm': '5105105105105100', 'ss': '4111111111111111', 'rw': '3566002020360505'}


Well, that looks promising. Let's say what the Luhn algorithm has to say!

In [20]:
[is_luhn_valid(v) for v in ccs_plain.values()]

[True, True, True, True, True]

Yay! Final step left: Decrypting the flag.

In [22]:
print('HV19{' + decrypt(flag) + '}')

HV19{5M113-420H4-KK3A1-19801}


Here's another thing which caused this challenge to be much more frustrating than it should've been: The `HV19{...}` pattern isn't part of the flag to decrypt, and the flag also isn't really recognizable as the solution (the only thing I can decipher is "5M113" = "smile"). This kind of thing only makes the challenge needlessly frustrating, because you might have the solution but not recognize it...